# Grain Process

In [1]:
import yt.units as u
import numpy as np
atoms = ['H', 'C', 'O']
mass = {'H': 1.0079, 'C': 12.0107, 'O': 15.9994}

## Grain model

In [12]:
r = 1000 * u.angstrom
V = 4 / 3 * np.pi * r**3
rho = 3 * u.gram / u.cm**3
Ns = 1e6
G_M_ratio = 100
nH = 2e4 / u.cm**3
T = 10 * u.Kelvin
ns = nH * u.mass_hydrogen / G_M_ratio / rho / V

## Accretion

$$R_{\mathrm{acc}}(i)=\overline{\sigma_{d}n_{d}}\langle v(i)\rangle n(i) = \sqrt{\frac{2 k_{B} T}{\pi m}}\sigma_{d}n_{d}n(i)$$

$$k_i= \sqrt{\frac{2 k_{B} T}{\pi m}}\sigma_{d}n_{d}n(i)=6.0557\sqrt{\frac{T\ (\text{K})}{\mu}}\times10^{-14}\text{ s}^{-1}$$

In [22]:
def mu(spe):
    t = ''
    m = 0
    for i in spe:
        if i.isdigit():
            m += mass[t] * (int(i) - 1)
        else:
            t = i
            m += mass[t]
    return(m)

def acc(spe, label=0):
    t = ''
    m = 0
    grain_spe = ''
    for i in spe:
        if i.isdigit():
            grain_spe += (t + '_dust') * (int(i) - 1)
            m += mass[t] * (int(i) - 1)
        else:
            t = i
            grain_spe += (t + '_dust')
            m += mass[t]
    react = '{},{},{},NONE,NONE,6.0557e-14*sqrt(Tgas/{})'.format(
        label, spe, grain_spe, m)
    return(react)

In [14]:
LIST_complex = ['H','O','OH','H2O','C','CH','CH2','CH3','CH3OH','CO','HCO']
for i, spe in enumerate(LIST_complex):
    print(acc(spe, label=i))

0,H,H_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/1.0079)
1,O,O_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/15.9994)
2,OH,O_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/17.0073)
3,C,C_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/12.0107)
4,CH,C_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/13.0186)
5,CH2,C_dustH_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/14.026499999999999)
6,CH3,C_dustH_dustH_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/15.0344)
7,CH3OH,C_dustH_dustH_dustH_dustO_dustH_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/32.0417)
8,CO,C_dustO_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/28.0101)
9,HCO,H_dustC_dustO_dust,NONE,NONE,6.0557e-14*sqrt(Tgas/29.018)


## Evaporation

### Thermal Evaporation

$$k_{\mathrm{eva}}(i)=\nu_{i} \exp \left[-E_{D} / kT\right]$$

$$\begin{aligned} \nu_{i} &=\left(2 \rho_{s} E_{D} / \pi^{2} m_{i}\right)^{1 / 2} \\ &=2.4 \times 10^{12} \mathrm{Hz}\left(\frac{\rho_{s}}{10^{15} \mathrm{cm}^{-2}}\right)^{1 / 2}\left(\frac{E_{D}}{350 \mathrm{K}}\right)^{1 / 2}\left(\frac{m_{i}}{m_{\mathrm{H}}}\right)^{-1 / 2} \end{aligned}$$

In [25]:
rhos = Ns/4/np.pi/r**2
np.sqrt(rhos.in_cgs()/1e15*u.cm**2)

0.8920620580763855 dimensionless

As a result,

$$k_{\mathrm{eva}}(i)=2.14 \times 10^{12} \mathrm{Hz}\left(\frac{E_{D}}{350 \mathrm{K}}\right)^{1 / 2}\left(\frac{m_{i}}{m_{\mathrm{H}}}\right)^{-1 / 2} \exp \left[-E_{D} / T\right]$$

In [24]:
calorie = 41858000*u.erg
kcal = 1e3*calorie
Eads = {}
Eads['H'] = 350*u.Kelvin
Eads['H2'] = 450*u.Kelvin
Eads['C'] = 800*u.Kelvin
Eads['O'] = 800*u.Kelvin
Eads['CH'] = 1.3*kcal/6.02e23/u.boltzmann_constant
Eads['OH'] = 2.5*kcal/6.02e23/u.boltzmann_constant
Eads['CO'] = 2.4*kcal/6.02e23/u.boltzmann_constant
Eads['CH2'] = 1.9*kcal/6.02e23/u.boltzmann_constant
Eads['H2O'] = 2.9*kcal/6.02e23/u.boltzmann_constant
Eads['CH3'] = 2.3*kcal/6.02e23/u.boltzmann_constant
Eads['CH3OH'] = 4.1*kcal/6.02e23/u.boltzmann_constant
for i in Eads.keys():
    print(2.14e12/u.s*np.sqrt(Eads[i]/350/u.Kelvin)/np.sqrt(mu(i)/mu('H'))*np.exp(-Eads[i]/10/u.Kelvin))

0.0013492949866714557 1/s
4.9115585387608356e-08 1/s
1.6915730418837247e-23 1/s
1.465625683254183e-23 1/s
3.0031875494947466e-17 1/s
2.0676307907048223e-43 1/s
2.429079066773433e-41 1/s
2.6348684610055793e-30 1/s
3.859307457976474e-52 1/s
4.9944413217040925e-39 1/s
1.952503538790247e-78 1/s
